In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_vi.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,919386,6011217110,4/14/20,252,PPP,"PSI TIRE SUPPLY, LLC","9010 Estate Cottage, Suite 2",CHRISTIANSTED,VI,820.0,...,999 Other,2382 Building Equipment Contractors,2382.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,250 to 499,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,NaN
1,919387,2616077208,4/16/20,252,PPP,VIRGIN ISLANDS TELEPHONE CORP,4611 Tutu Park #200,ST THOMAS,VI,802.0,...,999 Other,5179 Other Telecommunications,5179.0,PUERTO RICO AND USVI DISTRICT OFFICE,d $2-5 million,250 to 499,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,NaN
2,919388,8157738307,1/29/21,252,PPS,THE BUCCANEER INC.,5409 Reef Condos,Christiansted,VI,820.0,...,999 Other,7211 Traveler Accommodation,7211.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,NaN
3,919389,7079827106,4/14/20,252,PPP,THE BUCCANEER INC,5007 ESTATE SHOYS PLOT 7,CHRISTIANSTED,VI,820.0,...,999 Other,5615 Travel Arrangement and Reservation Services,5615.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,NaN
4,919390,4157848405,2/6/21,252,PPS,PROFESSIONAL DESIGN BUILDERS INC.,4028 Annas Retreat,St Thomas,VI,802.0,...,999 Other,2362 Nonresidential Building Construction,2362.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,919671,9409698408,2/17/21,252,PPS,ANGRY NATE'S BOATHOUSE & SEAFOOD EMPORIUM INC.,1201 King Cross St,Christiansted,VI,820.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1201 King Cross St, Christiansted, VI",17.744646,-64.704127,NaN
286,919672,8643378606,3/25/21,252,PPP,FLEMINGS' TRANSPORT CO.INC.,HENRY E ROHLSEN AIRPORT CARGO BLDG KINGSHILL V...,KINGSHILL,VI,850.0,...,999 Other,4841 General Freight Trucking,4841.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,HENRY E ROHLSEN AIRPORT CARGO BLDG KINGSHILL V...,17.699498,-64.797470,NaN
287,919673,4155348001,6/25/20,252,PPP,"ISLAND WIRING, LLC","6100 RED HOOK QTRS, #2, STE. C3-1",ST. THOMAS,VI,802.0,...,999 Other,8112 Electronic and Precision Equipment Repair...,8112.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6100 RED HOOK QTRS, #2, STE. C3-1, ST. THOMAS, VI",18.325982,-64.851845,NaN
288,919674,4251498806,4/16/21,252,PPS,CLINICAL LABORATORY INC.,Island Medical Center Suite 6,Christiansted,VI,820.0,...,999 Other,6215 Medical and Diagnostic Laboratories,6215.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Island Medical Center Suite 6, Christiansted, VI",17.746640,-64.703198,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
195,2087237800,"4ABC-1, HULL BAY, VI",NaN,NaN


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,6011217110,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198
1,2616077208,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935
2,8157738307,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198
3,7079827106,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198
4,4157848405,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("../data/state_data/geo/geo_fips/VI_fips_scraped.csv") 
fips_check_scraped.head() #15869

In [27]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [28]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

<h4>FIPS Script

In [6]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [7]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
#driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))
#executable_path = {'executable_path': webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))}
#executable_path = {'executable_path': ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()}
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('78')] #VI
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

The version of chrome cannot be detected. Trying with latest driver version


  0%|          | 0/289 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [8]:
geo_id_list =geo_id 

Length Check

In [9]:
print(len(geo_id_list))

289


Matching FIPS to DataFrame

In [10]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:25062]

In [11]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
0,6011217110,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,[780109702002030]
1,2616077208,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,[780309602003011]
2,8157738307,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,[780109702002030]
3,7079827106,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,[780109702002030]
4,4157848405,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,[780309602003011]
...,...,...,...,...,...
285,9409698408,"1201 King Cross St, Christiansted, VI",17.744646,-64.704127,[780109702002042]
286,8643378606,HENRY E ROHLSEN AIRPORT CARGO BLDG KINGSHILL V...,17.699498,-64.797470,[780109714002011]
287,4155348001,"6100 RED HOOK QTRS, #2, STE. C3-1, ST. THOMAS, VI",18.325982,-64.851845,[780309601001015]
288,4251498806,"Island Medical Center Suite 6, Christiansted, VI",17.746640,-64.703198,[780109702002030]


Remove brackets

In [12]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
0,6011217110,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,780109702002030
1,2616077208,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,780309602003011
2,8157738307,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,780109702002030
3,7079827106,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,780109702002030
4,4157848405,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,780309602003011
...,...,...,...,...,...
285,9409698408,"1201 King Cross St, Christiansted, VI",17.744646,-64.704127,780109702002042
286,8643378606,HENRY E ROHLSEN AIRPORT CARGO BLDG KINGSHILL V...,17.699498,-64.797470,780109714002011
287,4155348001,"6100 RED HOOK QTRS, #2, STE. C3-1, ST. THOMAS, VI",18.325982,-64.851845,780309601001015
288,4251498806,"Island Medical Center Suite 6, Christiansted, VI",17.746640,-64.703198,780109702002030


In [13]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z


Parse for Merge

In [14]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,6011217110,780109702002030
1,2616077208,780309602003011
2,8157738307,780109702002030
3,7079827106,780109702002030
4,4157848405,780309602003011
...,...,...
285,9409698408,780109702002042
286,8643378606,780109714002011
287,4155348001,780309601001015
288,4251498806,780109702002030


Read in base data

In [15]:
#if from geocode file
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_vi.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head()

#if from FIPS file
#fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/VI_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
#fips_check.head() #22826


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,919386,919386,919386,6011217110,4/14/20,252,PPP,"PSI TIRE SUPPLY, LLC","9010 Estate Cottage, Suite 2",CHRISTIANSTED,...,999 Other,2382 Building Equipment Contractors,2382.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,250 to 499,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,NaN
1,919387,919387,919387,2616077208,4/16/20,252,PPP,VIRGIN ISLANDS TELEPHONE CORP,4611 Tutu Park #200,ST THOMAS,...,999 Other,5179 Other Telecommunications,5179.0,PUERTO RICO AND USVI DISTRICT OFFICE,d $2-5 million,250 to 499,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,NaN
2,919388,919388,919388,8157738307,1/29/21,252,PPS,THE BUCCANEER INC.,5409 Reef Condos,Christiansted,...,999 Other,7211 Traveler Accommodation,7211.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,NaN
3,919389,919389,919389,7079827106,4/14/20,252,PPP,THE BUCCANEER INC,5007 ESTATE SHOYS PLOT 7,CHRISTIANSTED,...,999 Other,5615 Travel Arrangement and Reservation Services,5615.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,NaN
4,919390,919390,919390,4157848405,2/6/21,252,PPS,PROFESSIONAL DESIGN BUILDERS INC.,4028 Annas Retreat,St Thomas,...,999 Other,2362 Nonresidential Building Construction,2362.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,NaN


Merge with MainFrame

In [16]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,919386,919386,919386,6011217110,4/14/20,252,PPP,"PSI TIRE SUPPLY, LLC","9010 Estate Cottage, Suite 2",CHRISTIANSTED,...,2382 Building Equipment Contractors,2382.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,250 to 499,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,NaN,780109702002030
1,919387,919387,919387,2616077208,4/16/20,252,PPP,VIRGIN ISLANDS TELEPHONE CORP,4611 Tutu Park #200,ST THOMAS,...,5179 Other Telecommunications,5179.0,PUERTO RICO AND USVI DISTRICT OFFICE,d $2-5 million,250 to 499,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,NaN,780309602003011
2,919388,919388,919388,8157738307,1/29/21,252,PPS,THE BUCCANEER INC.,5409 Reef Condos,Christiansted,...,7211 Traveler Accommodation,7211.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,NaN,780109702002030
3,919389,919389,919389,7079827106,4/14/20,252,PPP,THE BUCCANEER INC,5007 ESTATE SHOYS PLOT 7,CHRISTIANSTED,...,5615 Travel Arrangement and Reservation Services,5615.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,NaN,780109702002030
4,919390,919390,919390,4157848405,2/6/21,252,PPS,PROFESSIONAL DESIGN BUILDERS INC.,4028 Annas Retreat,St Thomas,...,2362 Nonresidential Building Construction,2362.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,NaN,780309602003011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,919671,919671,919671,9409698408,2/17/21,252,PPS,ANGRY NATE'S BOATHOUSE & SEAFOOD EMPORIUM INC.,1201 King Cross St,Christiansted,...,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1201 King Cross St, Christiansted, VI",17.744646,-64.704127,NaN,780109702002042
286,919672,919672,919672,8643378606,3/25/21,252,PPP,FLEMINGS' TRANSPORT CO.INC.,HENRY E ROHLSEN AIRPORT CARGO BLDG KINGSHILL V...,KINGSHILL,...,4841 General Freight Trucking,4841.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,HENRY E ROHLSEN AIRPORT CARGO BLDG KINGSHILL V...,17.699498,-64.797470,NaN,780109714002011
287,919673,919673,919673,4155348001,6/25/20,252,PPP,"ISLAND WIRING, LLC","6100 RED HOOK QTRS, #2, STE. C3-1",ST. THOMAS,...,8112 Electronic and Precision Equipment Repair...,8112.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6100 RED HOOK QTRS, #2, STE. C3-1, ST. THOMAS, VI",18.325982,-64.851845,NaN,780309601001015
288,919674,919674,919674,4251498806,4/16/21,252,PPS,CLINICAL LABORATORY INC.,Island Medical Center Suite 6,Christiansted,...,6215 Medical and Diagnostic Laboratories,6215.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Island Medical Center Suite 6, Christiansted, VI",17.746640,-64.703198,NaN,780109702002030


<b>Fill FIPS Columns - if FIPS file

In [17]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,919386,919386,919386,6011217110,4/14/20,252,PPP,"PSI TIRE SUPPLY, LLC","9010 Estate Cottage, Suite 2",CHRISTIANSTED,...,2382.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,250 to 499,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,NaN,780109702002030,780109702002030
1,919387,919387,919387,2616077208,4/16/20,252,PPP,VIRGIN ISLANDS TELEPHONE CORP,4611 Tutu Park #200,ST THOMAS,...,5179.0,PUERTO RICO AND USVI DISTRICT OFFICE,d $2-5 million,250 to 499,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,NaN,780309602003011,780309602003011
2,919388,919388,919388,8157738307,1/29/21,252,PPS,THE BUCCANEER INC.,5409 Reef Condos,Christiansted,...,7211.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,NaN,780109702002030,780109702002030
3,919389,919389,919389,7079827106,4/14/20,252,PPP,THE BUCCANEER INC,5007 ESTATE SHOYS PLOT 7,CHRISTIANSTED,...,5615.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,NaN,780109702002030,780109702002030
4,919390,919390,919390,4157848405,2/6/21,252,PPS,PROFESSIONAL DESIGN BUILDERS INC.,4028 Annas Retreat,St Thomas,...,2362.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,NaN,780309602003011,780309602003011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,919671,919671,919671,9409698408,2/17/21,252,PPS,ANGRY NATE'S BOATHOUSE & SEAFOOD EMPORIUM INC.,1201 King Cross St,Christiansted,...,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1201 King Cross St, Christiansted, VI",17.744646,-64.704127,NaN,780109702002042,780109702002042
286,919672,919672,919672,8643378606,3/25/21,252,PPP,FLEMINGS' TRANSPORT CO.INC.,HENRY E ROHLSEN AIRPORT CARGO BLDG KINGSHILL V...,KINGSHILL,...,4841.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,HENRY E ROHLSEN AIRPORT CARGO BLDG KINGSHILL V...,17.699498,-64.797470,NaN,780109714002011,780109714002011
287,919673,919673,919673,4155348001,6/25/20,252,PPP,"ISLAND WIRING, LLC","6100 RED HOOK QTRS, #2, STE. C3-1",ST. THOMAS,...,8112.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6100 RED HOOK QTRS, #2, STE. C3-1, ST. THOMAS, VI",18.325982,-64.851845,NaN,780309601001015,780309601001015
288,919674,919674,919674,4251498806,4/16/21,252,PPS,CLINICAL LABORATORY INC.,Island Medical Center Suite 6,Christiansted,...,6215.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Island Medical Center Suite 6, Christiansted, VI",17.746640,-64.703198,NaN,780109702002030,780109702002030


Drop Extra FIPS Columns

In [18]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,919386,919386,919386,6011217110,4/14/20,252,PPP,"PSI TIRE SUPPLY, LLC","9010 Estate Cottage, Suite 2",CHRISTIANSTED,...,999 Other,2382 Building Equipment Contractors,2382.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,250 to 499,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,780109702002030
1,919387,919387,919387,2616077208,4/16/20,252,PPP,VIRGIN ISLANDS TELEPHONE CORP,4611 Tutu Park #200,ST THOMAS,...,999 Other,5179 Other Telecommunications,5179.0,PUERTO RICO AND USVI DISTRICT OFFICE,d $2-5 million,250 to 499,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,780309602003011
2,919388,919388,919388,8157738307,1/29/21,252,PPS,THE BUCCANEER INC.,5409 Reef Condos,Christiansted,...,999 Other,7211 Traveler Accommodation,7211.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,780109702002030
3,919389,919389,919389,7079827106,4/14/20,252,PPP,THE BUCCANEER INC,5007 ESTATE SHOYS PLOT 7,CHRISTIANSTED,...,999 Other,5615 Travel Arrangement and Reservation Services,5615.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,780109702002030
4,919390,919390,919390,4157848405,2/6/21,252,PPS,PROFESSIONAL DESIGN BUILDERS INC.,4028 Annas Retreat,St Thomas,...,999 Other,2362 Nonresidential Building Construction,2362.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,780309602003011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,919671,919671,919671,9409698408,2/17/21,252,PPS,ANGRY NATE'S BOATHOUSE & SEAFOOD EMPORIUM INC.,1201 King Cross St,Christiansted,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1201 King Cross St, Christiansted, VI",17.744646,-64.704127,780109702002042
286,919672,919672,919672,8643378606,3/25/21,252,PPP,FLEMINGS' TRANSPORT CO.INC.,HENRY E ROHLSEN AIRPORT CARGO BLDG KINGSHILL V...,KINGSHILL,...,999 Other,4841 General Freight Trucking,4841.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,HENRY E ROHLSEN AIRPORT CARGO BLDG KINGSHILL V...,17.699498,-64.797470,780109714002011
287,919673,919673,919673,4155348001,6/25/20,252,PPP,"ISLAND WIRING, LLC","6100 RED HOOK QTRS, #2, STE. C3-1",ST. THOMAS,...,999 Other,8112 Electronic and Precision Equipment Repair...,8112.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6100 RED HOOK QTRS, #2, STE. C3-1, ST. THOMAS, VI",18.325982,-64.851845,780309601001015
288,919674,919674,919674,4251498806,4/16/21,252,PPS,CLINICAL LABORATORY INC.,Island Medical Center Suite 6,Christiansted,...,999 Other,6215 Medical and Diagnostic Laboratories,6215.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Island Medical Center Suite 6, Christiansted, VI",17.746640,-64.703198,780109702002030


In [19]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
195,919581,919581,919581,2087237800,5/22/20,252,PPP,"FIRST RESORT PAINTING, INC.",4ABC-1,HULL BAY,...,999 Other,2383 Building Finishing Contractors,2383.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"4ABC-1, HULL BAY, VI",NaN,NaN,NaN


Convert to File

In [20]:
fips_VI = fips_merge_drop

In [21]:
fips_VI.to_csv("../../data/state_data/geo/geo_fips/VI_fips_scraped.csv") 

Review & Compare

In [22]:
geo_VI_fips = pd.read_csv("../../data/state_data/geo/geo_fips/VI_fips_scraped.csv") 
geo_VI_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,919386,919386,919386,6011217110,4/14/20,252,PPP,"PSI TIRE SUPPLY, LLC","9010 Estate Cottage, Suite 2",...,999 Other,2382 Building Equipment Contractors,2382.0,PUERTO RICO AND USVI DISTRICT OFFICE,e $5-10 million,250 to 499,"9010 Estate Cottage, Suite 2, CHRISTIANSTED, VI",17.746640,-64.703198,7.801097e+14
1,1,919387,919387,919387,2616077208,4/16/20,252,PPP,VIRGIN ISLANDS TELEPHONE CORP,4611 Tutu Park #200,...,999 Other,5179 Other Telecommunications,5179.0,PUERTO RICO AND USVI DISTRICT OFFICE,d $2-5 million,250 to 499,"4611 Tutu Park #200, ST THOMAS, VI",18.338310,-64.890935,7.803096e+14
2,2,919388,919388,919388,8157738307,1/29/21,252,PPS,THE BUCCANEER INC.,5409 Reef Condos,...,999 Other,7211 Traveler Accommodation,7211.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"5409 Reef Condos, Christiansted, VI",17.746640,-64.703198,7.801097e+14
3,3,919389,919389,919389,7079827106,4/14/20,252,PPP,THE BUCCANEER INC,5007 ESTATE SHOYS PLOT 7,...,999 Other,5615 Travel Arrangement and Reservation Services,5615.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"5007 ESTATE SHOYS PLOT 7, CHRISTIANSTED, VI",17.746640,-64.703198,7.801097e+14
4,4,919390,919390,919390,4157848405,2/6/21,252,PPS,PROFESSIONAL DESIGN BUILDERS INC.,4028 Annas Retreat,...,999 Other,2362 Nonresidential Building Construction,2362.0,PUERTO RICO AND USVI DISTRICT OFFICE,c $1-2 million,100 to 249,"4028 Annas Retreat, St Thomas, VI",18.340437,-64.891130,7.803096e+14


In [23]:
##### fill in missing Lat, Long, FIPS
geo_VI_fips.loc[geo_VI_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
195,195,919581,919581,919581,2087237800,5/22/20,252,PPP,"FIRST RESORT PAINTING, INC.",4ABC-1,...,999 Other,2383 Building Finishing Contractors,2383.0,PUERTO RICO AND USVI DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"4ABC-1, HULL BAY, VI",NaN,NaN,NaN


In [24]:
geo_VI_fips["BusinessType"] = geo_VI_fips["BusinessType"].fillna('Not Available')
geo_VI_fips["BusinessType"].unique()
geo_VI_fips.to_csv("../../data/state_data/geo/geo_fips/VI_fips_scraped.csv") 
geo_VI_fips =pd.read_csv("../../data/state_data/geo/geo_fips/VI_fips_scraped.csv") 
geo_VI_fips["BusinessType"].unique()

array(['Limited Liability Company(LLC)', 'Corporation',
       'Subchapter S Corporation', 'Cooperative',
       'Non-Profit Organization', 'Joint Venture',
       'Limited Liability Partnership', 'Professional Association',
       'Partnership', 'Sole Proprietorship'], dtype=object)